In [4]:
import os
import gspread
import pandas as pd
from config_loader import load_google_credentials


In [35]:
# src/config_loader.py
import os
from google.oauth2.service_account import Credentials

def load_google_credentials():
    # Caminho para o arquivo de credenciais
    cred_path = r"C:\Users\john.alencar\Documents\GitHub\programacao\ONBOARDING\config\credentials\automacao-onboarding-440312-791006d38824.json"
    
    # Verifica se o arquivo existe
    if not os.path.exists(cred_path):
        raise FileNotFoundError(f"Arquivo de credenciais não encontrado: {cred_path}")
    
    # Define os escopos necessários para acessar o Google Sheets e o Google Drive
    scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
    
    # Carrega as credenciais
    credentials = Credentials.from_service_account_file(cred_path, scopes=scopes)
    return credentials

In [36]:
def get_google_sheet_data(worksheet_name):
    # Carregar credenciais usando o módulo configurado
    credentials = load_google_credentials()
    
    # Autenticar e acessar o Google Sheets
    gc = gspread.authorize(credentials)

    # Abra a planilha e selecione uma aba
    spreadsheet = gc.open("Solicitações de Acesso - TI")
    worksheet = spreadsheet.worksheet(worksheet_name) 

    # Ler a coluna única da planilha (começando da segunda linha para evitar o cabeçalho)
    data = worksheet.col_values(1)[1:]  # Ignora o cabeçalho da primeira linha

    # Separar os dados por vírgula e criar um DataFrame
    data_split = [list(map(str.strip, row.split(','))) for row in data]  # Limpa os espaços
    columns = ['givenName', 'sn', 'userPrincipalName', 'sAMAccountName', 'displayName',
               'name/cn', 'description', 'physicalDeliveryOfficeName', 'mail', 'title', 'company']
    
    return pd.DataFrame(data_split, columns=columns)


In [40]:
google_sheet_df = get_google_sheet_data("Planilha3")

In [41]:
google_sheet_df.tail()

,givenName,sn,userPrincipalName,sAMAccountName,displayName,name/cn,description,physicalDeliveryOfficeName,mail,title,company
391,Milena,Vidal Do Nascimento,milena.nascimento,milena.nascimento,Milena Vidal Do Nascimento,Milena Vidal Do Nascimento,MEDIADOR ESCOLAR,EE Barra,milena.nascimento@escolaeleva.com.br,MEDIADOR ESCOLAR,EE Barra
392,Eliana,Andressa Martins Gomes,eliana.gomes,eliana.gomes,Eliana Andressa Martins Gomes,Eliana Andressa Martins Gomes,AUXILIAR DE PROFESSOR,EE Barra,eliana.gomes@escolaeleva.com.br,AUXILIAR DE PROFESSOR,EE Barra
393,Isabela,Franca Colmenero,isabela.colmenero,isabela.colmenero,Isabela Franca Colmenero,Isabela Franca Colmenero,BIBLIOTECÁRIA,EE Botafogo,isabela.colmenero@escolaeleva.com.br,BIBLIOTECÁRIA,EE Botafogo
394,Isabela2,Franca Colmenero,isabela.colmenero2,isabela.colmenero2,Isabela Franca Colmenero,Isabela Franca Colmenero,BIBLIOTECÁRIA,EE Botafogo,isabela.colmenero@escolaeleva.com.br,BIBLIOTECÁRIA,EE Botafogo
395,Isabela3,Franca Colmenero,isabela.colmenero3,isabela.colmenero3,Isabela Franca Colmenero,Isabela Franca Colmenero,BIBLIOTECÁRIA,EE Botafogo,isabela.colmenero@escolaeleva.com.br,BIBLIOTECÁRIA,EE Botafogo


In [18]:
def get_csv_data(csv_file_path):
    try:
        return pd.read_csv(csv_file_path)
    except FileNotFoundError:
        return pd.DataFrame()  # Retorna um DataFrame vazio se o arquivo não existir

In [20]:
# Função para adicionar novas linhas do CSV se houver alteração na planilha
def append_new_rows(google_sheet_df, csv_df):
    #chave para comparação
    key_columns = ['userPrincipalName']

    # Realiza um merge para identificar novas linhas
    merged_df = pd.merge(google_sheet_df, csv_df[key_columns], how="left", on=key_columns, indicator=True)

    # Encontrando novas linhas que não estão no CSV
    new_rows = merged_df[merged_df['_merge'] == 'left_only']

    # Adicionando novas linhas ao CSV
    if not new_rows.empty:
        new_rows.drop(columns=['_merge'], inplace=True)  # Remove a coluna de merge

        # Limpa os espaços nas linhas e remove colunas vazias
        new_rows = new_rows.apply(lambda x: x.str.strip(), axis=1)  # Limpa os espaços nas linhas
        new_rows.dropna(axis=1, how='all', inplace=True)  # Remove colunas vazias
        new_rows.dropna(how='all', inplace=True)  # Remove linhas vazias
        
        new_rows.head()
        # # Verifica se o CSV já existe
        # file_exists = os.path.exists(csv_file_path)
        
        # # Adiciona novas linhas ao CSV sem repetir o cabeçalho
        # new_rows.to_csv(csv_file_path, mode='a', header=not file_exists, index=False)

        print(f"{len(new_rows)} novas linhas adicionadas ao arquivo CSV")
    else:
        print("Não há novas linhas para adicionar")


In [26]:
# Nome da aba e caminho do arquivo CSV
worksheet_name = "Planilha3"
csv_file_path = r"C:\Users\john.alencar\Documents\GitHub\programacao\ONBOARDING\automacao\automacao copy.csv"

# Obtendo dados da planilha e do CSV
google_sheet_data = get_google_sheet_data(worksheet_name)
csv_df = get_csv_data(csv_file_path)

In [27]:

csv_df = get_csv_data(csv_file_path)
csv_df.head()

,givenName,sn,userPrincipalName,sAMAccountName,displayName,name/cn,description,physicalDeliveryOfficeName,mail,title,company
0,Fernando,Henrique Da Silva,fernando.silva,fernando.silva,Fernando Henrique Da Silva,Fernando Henrique Da Silva,AUXILIAR DE OPERAÇÕES,EE Recife,fernando.silva@escolaeleva.com.br,AUXILIAR DE OPERAÇÕES,EE Recife


In [32]:
#chave para comparação
key_columns = ['userPrincipalName']

# Realiza um merge para identificar novas linhas
merged_df = pd.merge(google_sheet_df, csv_df[key_columns], how="left", on=key_columns, indicator=True)

# Encontrando novas linhas que não estão no CSV
new_rows = merged_df[merged_df['_merge'] == 'left_only']

# Adicionando novas linhas ao CSV
if not new_rows.empty:
    new_rows.drop(columns=['_merge'], inplace=True)  # Remove a coluna de merge

    # Limpa os espaços nas linhas e remove colunas vazias
    new_rows = new_rows.apply(lambda x: x.str.strip(), axis=1)  # Limpa os espaços nas linhas
    new_rows.dropna(axis=1, how='all', inplace=True)  # Remove colunas vazias
    new_rows.dropna(how='all', inplace=True)  # Remove linhas vazias
    
    new_rows.head()
    # # Verifica se o CSV já existe
    # file_exists = os.path.exists(csv_file_path)
    
    # # Adiciona novas linhas ao CSV sem repetir o cabeçalho
    # new_rows.to_csv(csv_file_path, mode='a', header=not file_exists, index=False)

    #print(f"{len(new_rows)} novas linhas adicionadas ao arquivo CSV")
else:
    print("Não há novas linhas para adicionar")


C:\Users\john.alencar\AppData\Local\Temp\ipykernel_14584\480770266.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_rows.drop(columns=['_merge'], inplace=True)  # Remove a coluna de merge


In [33]:
new_rows.head()

,givenName,sn,userPrincipalName,sAMAccountName,displayName,name/cn,description,physicalDeliveryOfficeName,mail,title,company
1,Ana,Waymberg Goldman,ana.goldman,ana.goldman,Ana Waymberg Goldman,Ana Waymberg Goldman,GERENTE EXECUTIVO DE ADMISSÕES,Central,ana.goldman@inspirededu.com,GERENTE EXECUTIVO DE ADMISSÕES,Central
2,Bianca,Silva Simbras,bianca.simbras,bianca.simbras,Bianca Silva Simbras,Bianca Silva Simbras,JOVEM APRENDIZ,Os Batutinhas - Barra,bianca.simbras@osbatutinhas.com.br,JOVEM APRENDIZ,Os Batutinhas - Barra
3,Fernando,Marcus Duarte de Barros,fernando.duarte,fernando.duarte,Fernando Marcus Duarte de Barros,Fernando Marcus Duarte de Barros,GERENTE DE GENTE E GESTÃO,EE Botafogo,fernando.duarte@inspirededu.com,GERENTE DE GENTE E GESTÃO,EE Botafogo
4,Luiz,Felippe Landim Magalhaes,luiz.magalhaes,luiz.magalhaes,Luiz Felippe Landim Magalhaes,Luiz Felippe Landim Magalhaes,GERENTE DE OPERAÇÕES,EE Urca,luiz.magalhaes@escolaeleva.com.br,GERENTE DE OPERAÇÕES,EE Urca
5,Maxwell,Arcangelo de Santana,maxwell.santana,maxwell.santana,Maxwell Arcangelo de Santana,Maxwell Arcangelo de Santana,AGENTE EDUCACIONAL,EE Barra,maxwell.santana@escolaeleva.com.br,AGENTE EDUCACIONAL,EE Barra


In [ ]:
new_rows.to_csv(csv_file_path, mode='a', header=not file_exists, index=False)